# 27) Practical CUDA

Last Time:

- GPUs and CUDA
- Kernel syntax examples
- Thread hirerachy
- Memory

Today:

1. When to use a GPU?  
2. Practical CUDA  
3. Memory   

## 1. When to use a GPU?

* GPUs have 2-4x greater floating point and bandwidth peak for the watts
  * also for the \$ if you buy enterprise gear
  * better for the \$ if you buy gaming gear
* Step 1 is to assess workload and latency requirements

![](VecDot_CPU_vs_GPU_size.png)
![](VecDot_CPU_vs_GPU_time.png)

* Don't waste time with GPUs if
  * your problem size or time to solution requirements don't align
  * if the work you'd like to move to the GPU is not a bottleneck
  * if the computation cost will be dwarfed by moving data to/from the GPU
    * often you need to restructure so that caller passes in data already on the device
    * can require nonlocal refactoring
* Almost never: pick one kernel at a time and move it to the GPU
  * Real-world examples: DOE ACME/E3SM projects (to pick on one high-profile application) has basically done this for five years and it still doesn't help their production workloads so they bought a non-GPU machine
  

### Okay, okay, okay.  What if I have the right workload?

#### Terminology/Intro

* [An even easier introduction to CUDA](https://devblogs.nvidia.com/even-easier-introduction-cuda/)
* [CUDA Programming Model](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#programming-model)

* On the CPU, we have a thread with vector registers/instructions
* In CUDA, we write code inside a single vector lane ("confusingly" called a CUDA thread)
* To get inside the lane, we launch a **kernel** from the CPU using special syntax. For example:

```c
add<<<numBlocks, blockSize>>>(N, x, y);
```

* needs to be compiled using `nvcc` compiler
* Logically 1D/2D/3D rectangular tiled iteration space

![CUDA: grid of thread blocks](../img/grid-of-thread-blocks.png "CUDA: grid of thread blocks")


* There are [many](https://en.wikipedia.org/wiki/CUDA#Version_features_and_specifications) constraints and limitations to the iteration "grid"

![CUDA constraints](../img/cuda-constraints.png "CUDA constraints")

* Control flow for CUDA threads is nominally independent, but performance will be poor if you don't coordinate threads within each block.
  * Implicit coordination:
    * Memory coalescing
    * Organize your algorithm to limit "divergence"
  * Explicit coordination:
    * Shared memory
    * `__syncthreads()`
    * Warp shuffles
* We implement the kernel by using the `__global__` attribute
  * Visible from the CPU
  * Special [built-in variables](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#built-in-variables) are defined
    * `gridDim`: dimension of the grid
    * `blockIdx`: block index within the grid
    * `blockDim`: dimensions of the block
    * `threadIdx`: thread index within the block.
  * There is also `__device__`, which is callable from other device functions
  * Can use `__host__ __device__` to compile two versions

![CUDA indexing](../img/cuda_indexing.png "CUDA indexing")

#### How does this relate to the hardware?

* Each thread block is assigned to one **streaming multiprocessor (SM)**
* Executed in warps (number of hardware lanes)
* Multiple warps (from the same or different thread blocks) execute like "hyperthreads"

## 2. Practical CUDA  

### [CUDA Best Practices Guide](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html)

#### Occupancy

> Thread instructions are executed sequentially in CUDA, and, as a result, executing other warps when one warp is paused or stalled is the only way to hide latencies and keep the hardware busy. Some metric related to the number of active warps on a multiprocessor is therefore important in **determining how effectively the hardware is kept busy**. This metric is _occupancy_.  [emphasis added]

* Reality: occupancy is just one aspect, and often inversely correlated with keeping the hardware busy (and with performance).

> Occupancy is the ratio of the number of active warps per multiprocessor to the maximum number of possible active warps.

* If your kernel uses fewer registers/less shared memory, more warps can be scheduled.
* Register/shared memory usage is determined by the compiler.


Code example: 

```{literalinclude} ../cuda_codes/module7-3/add.cu
:language: cuda
:linenos: true
```

! nvcc ../cuda_codes/lecture7-3/add.cu --resource-usage -o add